This file contains


1.   New Version of lat long sampler which ensures all points sampled are strictly within boundary and follow the population density distribution
2.   Code for assigning lat longs to households generated by the IPU method
3.   Code for assigning wards to households so that the number of people in each ward is roughly similar to the actual values
4.   Code for assigning ages to people in buckets
5.   Code for regression of age on height and weight


This notebook has been cleaned so that the variables names are not dependent on any city. Next time we need a population, we just need to adjust the files being read and nothing else needs to be changed to generate the output from the notebook

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip3 install geopandas

In [ ]:
!wget https://data.worldpop.org/GIS/Population_Density/Global_2000_2020_1km/2020/IND/ind_pd_2020_1km_ASCII_XYZ.zip
!unzip ind_pd_2020_1km_ASCII_XYZ.zip

--2021-05-22 06:21:04--  https://data.worldpop.org/GIS/Population_Density/Global_2000_2020_1km/2020/IND/ind_pd_2020_1km_ASCII_XYZ.zip
Resolving data.worldpop.org (data.worldpop.org)... ::ffff:152.78.118.157, 152.78.118.157
Connecting to data.worldpop.org (data.worldpop.org)|::ffff:152.78.118.157|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53063505 (51M) [application/zip]
Saving to: ‘ind_pd_2020_1km_ASCII_XYZ.zip’

ind_pd_2020_1km_ASC 100%[===================>]  50.60M  1.43MB/s    in 33s     

2021-05-22 06:21:38 (1.52 MB/s) - ‘ind_pd_2020_1km_ASCII_XYZ.zip’ saved [53063505/53063505]

Archive:  ind_pd_2020_1km_ASCII_XYZ.zip
  inflating: ind_pd_2020_1km_ASCII_XYZ.csv  


In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import random
from shapely.geometry import Point, MultiPoint
from shapely.ops import unary_union

from tqdm import tqdm
tqdm.pandas()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [ ]:
population_density_data = pd.read_csv("ind_pd_2020_1km_ASCII_XYZ.csv")
columns_rename = {"X":"longitude", "Y":"latitude", "Z":"population_density"}
population_density_data['X'] = population_density_data['X'].round(6)
population_density_data['Y'] = population_density_data['Y'].round(6)
population_density_data.rename(columns_rename, axis=1, inplace=True)
population_density_data['point_object'] = population_density_data.progress_apply(lambda x : Point(x['longitude'], x['latitude']), axis=1)

100%|██████████| 4010402/4010402 [01:45<00:00, 38046.10it/s]


In [ ]:
population_density_data

,longitude,latitude,population_density,point_object
0,77.827916,35.503750,1.465800,POINT (77.827916 35.50375)
1,77.836250,35.503750,1.427846,POINT (77.83625000000001 35.50375)
2,77.844583,35.503750,0.473976,POINT (77.844583 35.50375)
3,77.819583,35.495417,1.207597,POINT (77.81958299999999 35.495417)
4,77.827916,35.495417,0.479271,POINT (77.827916 35.495417)
...,...,...,...,...
4010397,93.827916,6.762083,20.232878,POINT (93.827916 6.762083)
4010398,93.836250,6.762083,17.460630,POINT (93.83625000000001 6.762083)
4010399,93.819583,6.753750,7.082357,POINT (93.81958299999999 6.75375)
4010400,93.827916,6.753750,14.493382,POINT (93.827916 6.75375)


In [ ]:
def add_point(latitude, longitude):
    distances = pow(population_density_data['latitude']-latitude, 2) + pow(population_density_data['longitude']-longitude,2)
    sorted_df = population_density_data.loc[distances.sort_values().index]
    mean_population_density = sorted_df.iloc[:4]['population_density'].mean()
    
    new_row_index = len(population_density_data)
    
    population_density_data.at[new_row_index, 'longitude'] =  longitude
    population_density_data.at[new_row_index, 'latitude'] = latitude
    population_density_data.at[new_row_index, 'population_density'] = mean_population_density
    population_density_data.at[new_row_index, 'point_object'] = Point(longitude, latitude)

In [ ]:
def get_lat_long_samples(n, polygon):
    subset = population_density_data[population_density_data['point_object'].progress_apply(polygon.contains)]
    
    if(len(subset)==0):
        raise Exception("No points within the given polygon")
    
    sample = subset.sample(weights='population_density', n=(n*10), replace=True).copy()
    
    sample.reset_index(drop=True, inplace=True)
    
    sample['latitude'] = sample['latitude'] + np.random.uniform(-0.015, 0.015, size=sample.shape[0])
    
    sample['longitude'] = sample['longitude'] + np.random.uniform(-0.015, 0.015, size=sample.shape[0])
    
    points = sample.progress_apply(lambda x : Point(x['longitude'], x['latitude']), axis=1)
    
    contained = points.progress_apply(polygon.contains)
    
    return sample[contained][['longitude', 'latitude']].sample(n, replace=True).values

In [ ]:
admin_units = gpd.read_file("https://raw.githubusercontent.com/datameet/Municipal_Spatial_Data/master/Mumbai/BMC_Wards.geojson")
admin_units.sort_values(by='name', inplace=True)
admin_units.reset_index(drop=True, inplace=True)
admin_units

,gid,name,geometry
0,1,A,"MULTIPOLYGON (((72.84025 18.94881, 72.84030 18..."
1,2,B,"MULTIPOLYGON (((72.84456 18.96342, 72.84461 18..."
2,3,C,"MULTIPOLYGON (((72.83198 18.96174, 72.83197 18..."
3,4,D,"MULTIPOLYGON (((72.81873 18.96901, 72.81878 18..."
4,5,E,"MULTIPOLYGON (((72.84677 18.98183, 72.84658 18..."
5,8,F/N,"MULTIPOLYGON (((72.87091 19.05119, 72.87103 19..."
6,6,F/S,"MULTIPOLYGON (((72.85625 19.01059, 72.85630 19..."
7,9,G/N,"MULTIPOLYGON (((72.86699 19.05237, 72.86738 19..."
8,7,G/S,"MULTIPOLYGON (((72.82689 19.01942, 72.82691 19..."
9,18,H/E,"MULTIPOLYGON (((72.85932 19.08400, 72.85932 19..."


In [ ]:
for admin_unit in admin_units.iterrows():
    admin_unit_centroid = admin_unit[1]['geometry'].centroid
    add_point(admin_unit_centroid.y, admin_unit_centroid.x)

In [ ]:
ward_wise_pop = """Ward A	18.9337657	72.8364969	185014
Ward B	18.9614551	72.8333212	127290
Ward C	18.946123	72.8249009	166161
Ward D	18.9626147	72.813162	346866
Ward E	18.9718778	72.8313269	393286
Ward F North	19.0294197	72.8546058	529034
Ward F South	19.0058779	72.8396881	360972
Ward G North	19.023175	72.8434324	599039
Ward G South	19.0083734	72.8304087	377749
Ward H East	19.0851064	72.8445455	557239
Ward H West	19.0561063	72.8352939	307581
Ward K East	19.1200923	72.8523868	823885
Ward K West	19.1195001	72.844486	748688
Ward L	19.0704672	72.8790936	902225
Ward M East	19.0564771	72.9215464	807720
Ward M West	19.0611012	72.8993043	411893
Ward N	19.0839316	72.9064422	622853
Ward P North	19.1877853	72.8423072	941366
Ward P South	19.1626595	72.8464575	463507
Ward R Central	19.2311189	72.8558279	562162
Ward R North	19.1200923	72.8523868	431368
Ward R South	19.2039634	72.8453958	691229
Ward S	19.1390295	72.9304517	743783
Ward T	19.1756249	72.950922	341463""".split("\n")
admin_unit_wise_population = pd.DataFrame(list(map(lambda x : x.split("\t"), ward_wise_pop)), columns = ['Name', 'Latitude', 'Longitude', 'TOT_P'])
admin_unit_wise_population['Name'] = admin_unit_wise_population['Name'].apply(lambda x : x[x.find(" ")+1:].replace(" North", "/N").replace(" South", "/S").replace(" East", "/E").replace(" West", "/W").replace(" Central", "/C"))
admin_unit_wise_population.sort_values(by='Name', inplace=True)
admin_unit_wise_population.reset_index(drop=True, inplace=True)
admin_unit_wise_population['TOT_P'] = admin_unit_wise_population['TOT_P'].apply(int)
admin_unit_wise_population

,Name,Latitude,Longitude,TOT_P
0,A,18.9337657,72.8364969,185014
1,B,18.9614551,72.8333212,127290
2,C,18.946123,72.8249009,166161
3,D,18.9626147,72.813162,346866
4,E,18.9718778,72.8313269,393286
5,F/N,19.0294197,72.8546058,529034
6,F/S,19.0058779,72.8396881,360972
7,G/N,19.023175,72.8434324,599039
8,G/S,19.0083734,72.8304087,377749
9,H/E,19.0851064,72.8445455,557239


In [ ]:
admin_unit_wise_population['lower_limit'] = admin_unit_wise_population['TOT_P'].cumsum()-admin_unit_wise_population['TOT_P']
admin_unit_wise_population['upper_limit'] = admin_unit_wise_population['TOT_P'].cumsum()
admin_unit_wise_population

,Name,Latitude,Longitude,TOT_P,lower_limit,upper_limit
0,A,18.9337657,72.8364969,185014,0,185014
1,B,18.9614551,72.8333212,127290,185014,312304
2,C,18.946123,72.8249009,166161,312304,478465
3,D,18.9626147,72.813162,346866,478465,825331
4,E,18.9718778,72.8313269,393286,825331,1218617
5,F/N,19.0294197,72.8546058,529034,1218617,1747651
6,F/S,19.0058779,72.8396881,360972,1747651,2108623
7,G/N,19.023175,72.8434324,599039,2108623,2707662
8,G/S,19.0083734,72.8304087,377749,2707662,3085411
9,H/E,19.0851064,72.8445455,557239,3085411,3642650


In [ ]:
total_population = int(np.ceil(admin_unit_wise_population['TOT_P'].sum()/10000)*10000)
total_population

12450000

In [ ]:
city_households = pd.read_csv("/content/drive/MyDrive/syndata_ism/mumbai/base_population_data/mumbai_city_households2.csv")
suburban_households = pd.read_csv("/content/drive/MyDrive/syndata_ism/mumbai/base_population_data/mumbai_suburban_households2.csv")
synthetic_households = pd.concat([city_households,suburban_households])
synthetic_households.drop(synthetic_households.columns[0], axis=1, inplace=True)
synthetic_households.reset_index(drop=True, inplace=True)
synthetic_households.index = synthetic_households['household_id']
synthetic_households

,household_id,residence,geog,hhsize
household_id,,,,
2433018,2433018,urban,Mumbai City,hhsize_4
2571328,2571328,urban,Mumbai City,hhsize_4
2222736,2222736,urban,Mumbai City,hhsize_6
2116355,2116355,urban,Mumbai City,hhsize_6
2235639,2235639,urban,Mumbai City,hhsize_710
...,...,...,...,...
1490518,1490518,urban,Mumbai Suburban,hhsize_710
1800564,1800564,urban,Mumbai Suburban,hhsize_6
1672493,1672493,urban,Mumbai Suburban,hhsize_710


In [ ]:
city_population = pd.read_csv("/content/drive/MyDrive/syndata_ism/mumbai/base_population_data/mumbai_city_persons.csv")
suburban_population = pd.read_csv("/content/drive/MyDrive/syndata_ism/mumbai/base_population_data/mumbai_suburban_persons.csv")
synthetic_population = pd.concat([city_population,suburban_population])
synthetic_population.drop(synthetic_population.columns[0], axis=1, inplace=True)
synthetic_population.reset_index(drop=True, inplace=True)
synthetic_population

,mem_id,gender,age,literacy,religion,caste,residence,working,geog,household_id
0,1,male,40to44,literate,hindu,SC,urban,yes,Mumbai City,2097582
1,1,male,40to44,literate,hindu,SC,urban,yes,Mumbai City,2100136
2,1,male,40to44,literate,hindu,SC,urban,yes,Mumbai City,2103889
3,1,male,40to44,literate,hindu,SC,urban,yes,Mumbai City,2105417
4,1,male,40to44,literate,hindu,SC,urban,yes,Mumbai City,2105440
...,...,...,...,...,...,...,...,...,...,...
12488487,1,female,65to69,illiterate,hindu,other,urban,yes,Mumbai Suburban,2093191
12488488,1,female,65to69,illiterate,hindu,other,urban,yes,Mumbai Suburban,2093910
12488489,1,female,65to69,illiterate,hindu,other,urban,yes,Mumbai Suburban,2093979
12488490,1,female,65to69,illiterate,hindu,other,urban,yes,Mumbai Suburban,2094032


In [ ]:
synthetic_population['age'] = synthetic_population['age'].apply(lambda x : random.randint(80,95) if (x=="80p") else int(x.split("to")[0])) + np.random.randint(0,5,size=len(synthetic_population))
synthetic_population

,mem_id,gender,age,literacy,religion,caste,residence,working,geog,household_id
0,1,male,41,literate,hindu,SC,urban,yes,Mumbai City,2097582
1,1,male,41,literate,hindu,SC,urban,yes,Mumbai City,2100136
2,1,male,42,literate,hindu,SC,urban,yes,Mumbai City,2103889
3,1,male,44,literate,hindu,SC,urban,yes,Mumbai City,2105417
4,1,male,40,literate,hindu,SC,urban,yes,Mumbai City,2105440
...,...,...,...,...,...,...,...,...,...,...
12488487,1,female,65,illiterate,hindu,other,urban,yes,Mumbai Suburban,2093191
12488488,1,female,67,illiterate,hindu,other,urban,yes,Mumbai Suburban,2093910
12488489,1,female,65,illiterate,hindu,other,urban,yes,Mumbai Suburban,2093979
12488490,1,female,65,illiterate,hindu,other,urban,yes,Mumbai Suburban,2094032


In [ ]:
synthetic_households['hhsize'] = synthetic_population.groupby('household_id').size()

In [ ]:
synthetic_households

,household_id,residence,geog,hhsize
household_id,,,,
2433018,2433018,urban,Mumbai City,4
2571328,2571328,urban,Mumbai City,4
2222736,2222736,urban,Mumbai City,6
2116355,2116355,urban,Mumbai City,6
2235639,2235639,urban,Mumbai City,8
...,...,...,...,...
1490518,1490518,urban,Mumbai Suburban,10
1800564,1800564,urban,Mumbai Suburban,6
1672493,1672493,urban,Mumbai Suburban,8


In [ ]:
for admin_unit_wise_population_info in admin_unit_wise_population.iterrows():
    subset_index = (synthetic_households['hhsize'].cumsum()>=admin_unit_wise_population_info[1]['lower_limit']) & (synthetic_households['hhsize'].cumsum()<=admin_unit_wise_population_info[1]['upper_limit'])
    synthetic_households.loc[subset_index, 'AdminUnitName'] = admin_unit_wise_population_info[1]['Name']
    synthetic_households.loc[subset_index, 'AdminUnitLatitude'] = admin_unit_wise_population_info[1]['Latitude']
    synthetic_households.loc[subset_index, 'AdminUnitLongitude'] = admin_unit_wise_population_info[1]['Longitude']

In [ ]:
synthetic_households.dropna(inplace=True)
synthetic_households

,household_id,residence,geog,hhsize,AdminUnitName,AdminUnitLatitude,AdminUnitLongitude
household_id,,,,,,,
2433018,2433018,urban,Mumbai City,4,A,18.9337657,72.8364969
2571328,2571328,urban,Mumbai City,4,A,18.9337657,72.8364969
2222736,2222736,urban,Mumbai City,6,A,18.9337657,72.8364969
2116355,2116355,urban,Mumbai City,6,A,18.9337657,72.8364969
2235639,2235639,urban,Mumbai City,8,A,18.9337657,72.8364969
...,...,...,...,...,...,...,...
210335,210335,urban,Mumbai Suburban,8,T,19.1756249,72.950922
1903782,1903782,urban,Mumbai Suburban,8,T,19.1756249,72.950922
1956510,1956510,urban,Mumbai Suburban,8,T,19.1756249,72.950922


In [ ]:
synthetic_households[['H_Lat', 'H_Lon']] = None
synthetic_households

,household_id,residence,geog,hhsize,AdminUnitName,AdminUnitLatitude,AdminUnitLongitude,H_Lat,H_Lon
household_id,,,,,,,,,
2433018,2433018,urban,Mumbai City,4,A,18.9337657,72.8364969,None,None
2571328,2571328,urban,Mumbai City,4,A,18.9337657,72.8364969,None,None
2222736,2222736,urban,Mumbai City,6,A,18.9337657,72.8364969,None,None
2116355,2116355,urban,Mumbai City,6,A,18.9337657,72.8364969,None,None
2235639,2235639,urban,Mumbai City,8,A,18.9337657,72.8364969,None,None
...,...,...,...,...,...,...,...,...,...
210335,210335,urban,Mumbai Suburban,8,T,19.1756249,72.950922,None,None
1903782,1903782,urban,Mumbai Suburban,8,T,19.1756249,72.950922,None,None
1956510,1956510,urban,Mumbai Suburban,8,T,19.1756249,72.950922,None,None


In [ ]:
for admin_unit_name in synthetic_households['AdminUnitName'].unique():
    print(admin_unit_name)
    admin_unit_polygon = admin_units[admin_units['name']==admin_unit_name]['geometry'].iloc[0]
    admin_unit_houses_index = synthetic_households['AdminUnitName']==admin_unit_name
    n_houses_in_admin_unit = len(synthetic_households[admin_unit_houses_index])
    points = get_lat_long_samples(n_houses_in_admin_unit, admin_unit_polygon)
    synthetic_households.loc[admin_unit_houses_index, ['H_Lon', 'H_Lat']] = points

  0%|          | 0/4010426 [00:00<?, ?it/s]

A


  0%|          | 7484/4010426 [00:00<00:53, 74839.89it/s]

B


  0%|          | 12106/4010426 [00:00<00:33, 121057.81it/s]

...

  0%|          | 9804/4010426 [00:00<00:40, 98038.69it/s]

S


  0%|          | 7758/4010426 [00:00<00:51, 77575.08it/s]

T


100%|██████████| 409340/409340 [00:37<00:00, 10785.59it/s]


In [ ]:
synthetic_households.index.name = 'hh_index'
synthetic_households

,household_id,residence,geog,hhsize,AdminUnitName,AdminUnitLatitude,AdminUnitLongitude,H_Lat,H_Lon
hh_index,,,,,,,,,
2433018,2433018,urban,Mumbai City,4,A,18.9337657,72.8364969,18.9393,72.8285
2571328,2571328,urban,Mumbai City,4,A,18.9337657,72.8364969,18.9389,72.8425
2222736,2222736,urban,Mumbai City,6,A,18.9337657,72.8364969,18.919,72.8226
2116355,2116355,urban,Mumbai City,6,A,18.9337657,72.8364969,18.9218,72.8322
2235639,2235639,urban,Mumbai City,8,A,18.9337657,72.8364969,18.938,72.8309
...,...,...,...,...,...,...,...,...,...
210335,210335,urban,Mumbai Suburban,8,T,19.1756249,72.950922,19.1673,72.9668
1903782,1903782,urban,Mumbai Suburban,8,T,19.1756249,72.950922,19.1685,72.9422
1956510,1956510,urban,Mumbai Suburban,8,T,19.1756249,72.950922,19.1791,72.9388


In [ ]:
columns_to_join = ['household_id', 'H_Lat', 'H_Lon', 'AdminUnitName', 'AdminUnitLatitude', 'AdminUnitLongitude']
merged_df = pd.merge(synthetic_population, synthetic_households[columns_to_join] ,on='household_id')
merged_df

,mem_id,gender,age,literacy,religion,caste,residence,working,geog,household_id,H_Lat,H_Lon,AdminUnitName,AdminUnitLatitude,AdminUnitLongitude
0,1,male,41,literate,hindu,SC,urban,yes,Mumbai City,2097582,18.9866,72.8527,F/S,19.0058779,72.8396881
1,1,male,44,literate,hindu,SC,urban,yes,Mumbai City,2097582,18.9866,72.8527,F/S,19.0058779,72.8396881
2,2,female,39,literate,hindu,SC,urban,no,Mumbai City,2097582,18.9866,72.8527,F/S,19.0058779,72.8396881
3,2,female,36,literate,hindu,SC,urban,no,Mumbai City,2097582,18.9866,72.8527,F/S,19.0058779,72.8396881
4,3,female,15,literate,hindu,SC,urban,no,Mumbai City,2097582,18.9866,72.8527,F/S,19.0058779,72.8396881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12442367,1,female,65,illiterate,hindu,other,urban,yes,Mumbai Suburban,2093979,19.1995,72.816,P/N,19.1877853,72.8423072
12442368,1,female,65,illiterate,hindu,other,urban,yes,Mumbai Suburban,2094032,19.1426,72.8101,K/W,19.1195001,72.844486
12442369,1,female,65,illiterate,hindu,other,urban,yes,Mumbai Suburban,2094032,19.1426,72.8101,K/W,19.1195001,72.844486
12442370,1,female,69,illiterate,hindu,other,urban,yes,Mumbai Suburban,2094072,19.2074,72.8528,R/S,19.2039634,72.8453958


In [ ]:
merged_df.to_csv("/content/drive/MyDrive/syndata_ism/mumbai/base_population_data/with_hlat_hlong.csv")